### Data sets were acquired from https://www.kaggle.com/c/titanic

In [1]:
import pandas as pd
import numpy as np
import sidetable
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline

df_train = pd.read_csv('train.csv', index_col=['PassengerId'])
test = pd.read_csv('test.csv', index_col=['PassengerId'])

In [2]:
# Have a look at the first five rows
df_train.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df_train.shape

(891, 11)

In [4]:
def drop_irrelevant(df):
    return df.drop(columns=['Name', 'Ticket', 'Cabin'])
df_train = drop_irrelevant(df_train)
X_test = drop_irrelevant(test)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,
892,3,male,34.5,0,0,7.8292,Q
893,3,female,47.0,1,0,7.0000,S
894,2,male,62.0,0,0,9.6875,Q
895,3,male,27.0,0,0,8.6625,S
896,3,female,22.0,1,1,12.2875,S
...,...,...,...,...,...,...,...
1305,3,male,NaN,0,0,8.0500,S
1306,1,female,39.0,0,0,108.9000,C
1307,3,male,38.5,0,0,7.2500,S


In [5]:
# Get X_train and y_train
y_train = df_train.Survived.copy()
X_train = df_train.drop(columns='Survived').copy()

In [6]:
# Retrive num_cols and cat_cols
num_cols = list(X_train._get_numeric_data().columns)
cat_cols = list(set(X_train.columns) - set(num_cols))

In [7]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer, make_column_selector

cat_pipe = make_pipeline(
            (SimpleImputer(strategy='most_frequent')),
            (OneHotEncoder(drop='first', handle_unknown='error'))
            )
num_pipe = make_pipeline(
            (IterativeImputer()),
            (StandardScaler())
            )            
preprocess_pipeline = make_column_transformer(
            (cat_pipe, cat_cols),
            (num_pipe, num_cols)
            )

In [8]:
preprocessor =  preprocess_pipeline.fit(X_train)
X_train = preprocessor.transform(X_train)
X_test = preprocessor.transform(X_test)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, stratify=y_train)

In [10]:
'''
Credit source: 
    https://www.tensorflow.org/tutorials/keras/keras_tuner
    https://github.com/keras-team/keras-tuner/blob/master/examples/cifar10.py
'''

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam

# Create the keras tuner model.
def build_model(hp):
    hp_drop_out = hp.Float('dropout', 0, 0.5, step=0.1)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    model = Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(Dense(units=hp.Int('units_' + str(i), min_value=32, max_value=512, step=32), activation='relu'))
#         model.add(BatchNormalization())
        model.add(Dropout(hp_drop_out))
#     model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    opt = Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4]))
    model.compile(optimizer=Adam(learning_rate=hp_learning_rate),
                loss='binary_crossentropy',
                metrics=['accuracy'])
    return model

import kerastuner as kt

tuner = kt.Hyperband(build_model,
                     objective='val_accuracy', 
                     max_epochs=16,
                     overwrite=False)

INFO:tensorflow:Reloading Oracle from existing project .\untitled_project\oracle.json
INFO:tensorflow:Reloading Tuner from .\untitled_project\tuner0.json


In [11]:
es = EarlyStopping(monitor='val_accuracy', verbose=0, patience=10, min_delta=1e-3)
tuner.search(X_train, y_train, epochs=64, batch_size=32, validation_data=(X_val, y_val), callbacks=[es])

Trial 25 Complete [00h 00m 10s]
val_accuracy: 0.8656716346740723

Best val_accuracy So Far: 0.8656716346740723
Total elapsed time: 00h 02m 19s
INFO:tensorflow:Oracle triggered exit


In [12]:
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train, y_train, epochs=64, validation_data=(X_val, y_val))

Epoch 1/64
24/24 [==============================] - 2s 22ms/step - loss: 0.6677 - accuracy: 0.5849 - val_loss: 0.5685 - val_accuracy: 0.7239
Epoch 2/64
24/24 [==============================] - ETA: 0s - loss: 0.5353 - accuracy: 0.74 - 0s 13ms/step - loss: 0.5318 - accuracy: 0.7518 - val_loss: 0.4938 - val_accuracy: 0.7537
Epoch 3/64
24/24 [==============================] - 0s 11ms/step - loss: 0.4569 - accuracy: 0.7907 - val_loss: 0.4128 - val_accuracy: 0.8358
Epoch 4/64
24/24 [==============================] - 0s 10ms/step - loss: 0.4208 - accuracy: 0.8163 - val_loss: 0.4559 - val_accuracy: 0.8060
Epoch 5/64
24/24 [==============================] - 0s 11ms/step - loss: 0.4299 - accuracy: 0.8357 - val_loss: 0.4113 - val_accuracy: 0.8582
Epoch 6/64
24/24 [==============================] - 0s 11ms/step - loss: 0.4214 - accuracy: 0.8346 - val_loss: 0.4392 - val_accuracy: 0.8284
Epoch 7/64
24/24 [==============================] - 0s 10ms/step - loss: 0.4443 - accuracy: 0.8248 - val_loss: 0

In [13]:
val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Best epoch: 57


In [14]:
# Re-instantiate the hypermodel and train it with the optimal number of epochs from above.
hypermodel = tuner.hypermodel.build(best_hps)
# Retrain the model
hypermodel.fit(X_train, y_train, epochs=best_epoch)

Epoch 1/57
24/24 [==============================] - 2s 6ms/step - loss: 0.6508 - accuracy: 0.5769
Epoch 2/57
24/24 [==============================] - 0s 7ms/step - loss: 0.5104 - accuracy: 0.8074
Epoch 3/57
24/24 [==============================] - 0s 8ms/step - loss: 0.4794 - accuracy: 0.7940
Epoch 4/57
24/24 [==============================] - 0s 8ms/step - loss: 0.4546 - accuracy: 0.8111
Epoch 5/57
24/24 [==============================] - 0s 6ms/step - loss: 0.4277 - accuracy: 0.8277
Epoch 6/57
24/24 [==============================] - 0s 6ms/step - loss: 0.4216 - accuracy: 0.8307
Epoch 7/57
24/24 [==============================] - 0s 8ms/step - loss: 0.4247 - accuracy: 0.8406
Epoch 8/57
24/24 [==============================] - 0s 8ms/step - loss: 0.4112 - accuracy: 0.8307
Epoch 9/57
24/24 [==============================] - 0s 7ms/step - loss: 0.4995 - accuracy: 0.7883
Epoch 10/57
24/24 [==============================] - 0s 6ms/step - loss: 0.4842 - accuracy: 0.8104
Epoch 11/57
24/24 [

In [15]:
eval_result = hypermodel.evaluate(X_val, y_val)
print("[test loss, test accuracy]:", eval_result)

5/5 [==============================] - 0s 2ms/step - loss: 0.5830 - accuracy: 0.8209
[test loss, test accuracy]: [0.5830051898956299, 0.8208954930305481]


In [16]:
# Retrain the model
hypermodel.fit(X_train, y_train, epochs=best_epoch)

Epoch 1/57
24/24 [==============================] - 0s 7ms/step - loss: 0.3372 - accuracy: 0.8573
Epoch 2/57
24/24 [==============================] - 0s 7ms/step - loss: 0.3355 - accuracy: 0.8481
Epoch 3/57
24/24 [==============================] - 0s 6ms/step - loss: 0.3268 - accuracy: 0.8613
Epoch 4/57
24/24 [==============================] - 0s 6ms/step - loss: 0.3336 - accuracy: 0.8639
Epoch 5/57
24/24 [==============================] - ETA: 0s - loss: 0.3360 - accuracy: 0.86 - 0s 7ms/step - loss: 0.3315 - accuracy: 0.8666
Epoch 6/57
24/24 [==============================] - 0s 7ms/step - loss: 0.3328 - accuracy: 0.8679
Epoch 7/57
24/24 [==============================] - 0s 6ms/step - loss: 0.3296 - accuracy: 0.8600
Epoch 8/57
24/24 [==============================] - 0s 6ms/step - loss: 0.3175 - accuracy: 0.8679
Epoch 9/57
24/24 [==============================] - 0s 6ms/step - loss: 0.3263 - accuracy: 0.8626
Epoch 10/57
24/24 [==============================] - 0s 7ms/step - loss: 0.3

In [17]:
# from keras.models import save_model
# hypermodel.save('best_model.h5')

In [18]:
# Load the model and predict
# from keras.models import load_model
# hypermodel = load_model('best_model.h5')
y_pred = hypermodel.predict(X_test)
y_pred = y_pred.astype(int)

In [19]:
y_pred = pd.DataFrame(y_pred, columns=['Survived'], index=test.index)
# saving the dataframe 
y_pred.to_csv('Predictions.csv') 

AttributeError: 'numpy.ndarray' object has no attribute 'index'

In [ ]:
import winsound
duration = 100  # milliseconds
freq = 3000  # Hz
winsound.Beep(freq, duration)